In [359]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn
%pip install zemberek-python
%pip install trnltk
%pip install git+https://github.com/emres/turkish-deasciifier.git
%pip install transformers datasets
%pip install torch
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/emres/turkish-deasciifier.git to c:\users\cemke\appdata\local\temp\pip-req-build-8jnsextu
  Resolved https://github.com/emres/turkish-deasciifier.git to commit 665154c734b09485c3d11ce0038cd121dd109594
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may 

  Running command git clone --filter=blob:none --quiet https://github.com/emres/turkish-deasciifier.git 'C:\Users\cemke\AppData\Local\Temp\pip-req-build-8jnsextu'


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [360]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import logging
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as st
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
from trnlp import TrnlpWord
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, TextClassificationPipeline
from transformers import pipeline
from trnlp import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [383]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()
            
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]   
            #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.full_text for tweet in tweets]
            #DataSet['Tweet_Degistirilmis'] = [tweet.full_text for tweet in tweets]
            #DataSet['Tweet_Ceviri']=[tweet.full_text for tweet in tweets]            
            #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
                
            #DataSet['Dil'] = [tweet.lang for tweet in tweets]
            DataSet.to_csv('tw.csv',index=False)
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan+"-filter:retweets", result_type="new", lang='tr',tweet_mode="extended")
            results=[]
            for item in cursor.items(20):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)            
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            Dataset=pd.read_csv('tw.csv')                   
            getTweets.TweetClear(DataSet)
            #getTweets.SentenceNormalization(DataSet)
            getTweets.TweetTranslate(DataSet)
            getTweets.WordSeparation(DataSet) 
            getTweets.ToxicSentence(DataSet) 
            getTweets.TrWordSeparation(DataSet)
            getTweets.TextblobSentiment(DataSet)           
            DataSet.to_csv('tw.csv',index=False)

    def TextblobSentiment(DataSet):
        try:
            data=DataSet
            print ("Duygu Analizi Yapılıyor...")            
            data["Tür"] = data["Tweet_Ayrı"].apply(getTweets.getPolarity)
            data["Yoğunluk"] = data["Tweet_Ayrı"].apply(getTweets.getSubjectivity)
            data.to_csv('textblob_eng_duygu.csv')
            print("Duygu Analizi Yapıldı!")                        
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    def TrWordSeparation(DataSet):
        try:
            data=DataSet
            print("Türkçe Kelime Ayrıştırma Yapılıyor...")
            data["Tweet_Ayrı_TR"]=data["Tweet_Degistirilmis"]
            for index in range(len(data["Tweet_Ayrı_TR"])):
                data["Tweet_Ayrı_TR"][index] = simple_token(data["Tweet_Ayrı_TR"][index], sw=stopwords)
            print("Türkçe Kelime Ayrıştırma Yapıldı!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TweetClear(DataSet):
        try:
            data=DataSet
            print ("Tweetler Temizleniyor...")
            data["Tweet_Degistirilmis"]=data["Tweet"]
            pattern = re.compile('http[s]?://(?:[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+|#[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|$[a-zA-ZğüşöçıİĞÜŞÖÇ]+|@[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ_]+|[,.^_$*%-;鶯!?’"“”:=|…#]+|\n+|RT[\s]+|amp[\s]+|gt[\s]')
            for index in range(len(data["Tweet_Degistirilmis"])):        
                data["Tweet_Degistirilmis"][index]=pattern.sub(' ', data["Tweet_Degistirilmis"][index])    
                data["Tweet_Degistirilmis"][index]=data["Tweet_Degistirilmis"][index].lower()              
            data["Tweet_Degistirilmis"]=data["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))  
                              
    def TweetTranslate(DataSet):
        try:
            data=DataSet
            print ("Tweetler Çeviriliyor...")
            data["Tweet_Ceviri"]=data["Tweet_Degistirilmis"]
            for index in range(len(data["Tweet_Ceviri"])):
                Ceviri_Blob=TextBlob(data["Tweet_Ceviri"][index])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                data["Tweet_Ceviri"][index]=Eng_Blob          
            print ("Tweetler Çevirildi!")                      
            return data        
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    def SentenceNormalization(DataSet):
        try:
            data=DataSet
            print("Cümle Normalizasyonu Yapılıyor...")
            morphology = TurkishMorphology.create_with_defaults()
            normalizer = TurkishSentenceNormalizer(morphology)
            for index in range(len(data["Tweet_Degistirilmis"])):                
                data["Tweet_Degistirilmis"][index]= normalizer.normalize(data["Tweet_Degistirilmis"][index])
            print("Cümle Normalizasyonu Yapıldı!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))
            
    def WordSeparation(DataSet):
        try:
            data=DataSet
            print("İngilizce Kelime Ayrıştırma Yapılıyor...")
            stop = st.words('english')
            data["Tweet_Ayrı"] = data["Tweet_Ceviri"].str.split()
            data["Tweet_Ayrı"] = data["Tweet_Ayrı"].apply(lambda x: [item for item in x if item not in stop])
            data["Tweet_Ayrı"] = data["Tweet_Ayrı"].str.join(" ")
            print("İngilizce Kelime Ayrıştırma Yapıldı!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))
    
    def ToxicSentence(DataSet):
        try:
            data=DataSet
            print("Toxic Cümle Analizi Yapılıyor...")
            model_path ="martin-ha/toxic-comment-model"
            tokenizer=AutoTokenizer.from_pretrained(model_path)
            model=AutoModelForSequenceClassification.from_pretrained(model_path)
            pipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)
            data["Tweet_Toxic"]=data["Tweet_Ceviri"]
            for index in range(len(data["Tweet_Toxic"])):
                sonuc=pipeline(str(data["Tweet_Toxic"][index]))
                if sonuc[0]['label']=="toxic":
                    data["Tweet_Toxic"][index]="Toxic"
                else:
                    data["Tweet_Toxic"][index]="Toxic değil"    
            print("Toxic Cümle Analizi Yapıldı!")
            return data  
        except Exception as err:
            print('Error: {}'.format(err))
        
    def getSubjectivity(DataSet):
        try: 
            sub=TextBlob(DataSet).sentiment.subjectivity               
            if sub >=0.75:
                return "%75 - %100"
            elif sub >=0.5 and sub <0.75:
                return "%50 - %75"
            elif sub >=0.25 and sub <0.50:
                return "%25 - %50"
            else:
                return "0 - 25%"
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getPolarity(DataSet):
        try:
            pol=TextBlob(DataSet).sentiment.polarity
            if pol>0.1:
                return "Pozitif"
            elif pol<-0.1:
                return "Negatif"
            else:
                return "Nötr"
        except Exception as err:
            print('Error: {}'.format(err))

In [384]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)

aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Tweetler Çeviriliyor...
Tweetler Çevirildi!
İngilizce Kelime Ayrıştırma Yapılıyor...
İngilizce Kelime Ayrıştırma Yapıldı!
Toxic Cümle Analizi Yapılıyor...
Toxic Cümle Analizi Yapıldı!
Türkçe Kelime Ayrıştırma Yapılıyor...
Türkçe Kelime Ayrıştırma Yapıldı!
Duygu Analizi Yapılıyor...
Duygu Analizi Yapıldı!


In [385]:
Veri=pd.read_csv('textblob_eng_duygu.csv')
Veri.head(10)

,Unnamed: 0,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tweet_Toxic,Tweet_Ayrı_TR,Tür,Yoğunluk
0,0,Mr Brown,2023-01-13 00:27:39+00:00,@kaansezyum Kot farkı mı kanky,kot farkı mı kanky,Is it a difference in jewah kanky,Is difference jewah kanky,Toxic değil,"['kot', 'farkı', 'kanky']",Nötr,0 - 25%
1,1,Merve Hazer Göksu,2023-01-13 00:24:22+00:00,Bakın arkadaşlar bu erkeklere yaranamazsınız. Az önce kot gömlek uyuyan kocayı kaldırıp rahat bişeyler giydirdim diye azar yedim. Sıçarım öyle uyku sersemliğine,bakın arkadaşlar bu erkeklere yaranamazsınız az önce kot gömlek uyuyan kocayı kaldırıp rahat bişeyler giydirdim diye azar yedim sıçarım öyle uyku sersemliğine,"Look, friends, you can't help these men, just a jeans shirt, I lifted the sleeping husband and dressed something comfortable, I ate it.","Look, friends, can't help men, jeans shirt, I lifted sleeping husband dressed something comfortable, I ate it.",Toxic değil,"['bakın', 'arkadaşlar', 'erkeklere', 'yaranamazsınız', 'kot', 'gömlek', 'uyuyan', 'kocayı', 'kaldırıp', 'rahat', 'bişeyler', 'giydirdim', 'azar', 'yedim', 'sıçarım', 'uyku', 'sersemliğine']",Pozitif,%75 - %100
2,2,🇹🇷meftunnn🇹🇷☝️,2023-01-12 23:01:06+00:00,@_efe_derya_ @cemonurhak Kot kafalı it,kot kafalı it,jeans,jeans,Toxic değil,"['kot', 'kafalı', 'it']",Nötr,0 - 25%
3,3,Dr.Life 🇹🇷,2023-01-12 22:29:29+00:00,"@nafiasybyrktr Aklı çok parası kot biri lazım olursa я тут , I am here 😂 ben buradayım",aklı çok parası kot biri lazım olursa я тут i am here ben buradayım,"If someone is needed to have a lot of money, I am here","If someone needed lot money, I",Toxic değil,"['aklı', 'parası', 'kot', 'lazım', 'olursa', 'я', 'тут', 'i', 'am', 'here', 'buradayım']",Nötr,0 - 25%
4,4,AksiPense,2023-01-12 22:13:17+00:00,@aylin_kotil @Canan_Kaftanci Beraat olunca adalet ceza kesinleşince ülkede adalet yok . Kot il,beraat olunca adalet ceza kesinleşince ülkede adalet yok kot il,"When it becomes acquitted, justice is finalized when the penalty is finalized, no justice in the country","When becomes acquitted, justice finalized penalty finalized, justice country",Toxic değil,"['beraat', 'olunca', 'adalet', 'ceza', 'kesinleşince', 'ülkede', 'adalet', 'yok', 'kot', 'il']",Nötr,0 - 25%
5,5,Trol Diker,2023-01-12 22:01:36+00:00,"@odatv Kafan da siyasal simgen çaput olsa ne olur AMK ! 5 Kilo makyaj suratta ,daracık kot götte ama saçın kılı gözükmesin !",kafan da siyasal simgen çaput olsa ne olur amk kilo makyaj suratta daracık kot götte ama saçın kılı gözükmesin,"What happens if your head is a political symbol, but the weight makeup in the face is tight jeans in the ass, but your hair does not appear.","What happens head political symbol, weight makeup face tight jeans ass, hair appear.",Toxic değil,"['kafan', 'siyasal', 'simgen', 'çaput', 'olsa', 'olur', 'amk', 'kilo', 'makyaj', 'suratta', 'daracık', 'kot', 'götte', 'saçın', 'kılı', 'gözükmesin']",Nötr,0 - 25%
6,6,semih,2023-01-12 21:51:41+00:00,Gerek fiziksel gerek mental olarak çok yorgun bir şekilde eve geldim şimdi kot pantolonla 15 saat uyuma vakti,gerek fiziksel gerek mental olarak çok yorgun bir şekilde eve geldim şimdi kot pantolonla saat uyuma vakti,"I came home very tired both physically and mentally, now it's time to sleep with jeans.","I came home tired physically mentally, time sleep jeans.",Toxic değil,"['gerek', 'fiziksel', 'gerek', 'mental', 'olarak', 'yorgun', 'şekilde', 'eve', 'geldim', 'kot', 'pantolonla', 'saat', 'uyuma', 'vakti']",Negatif,%25 - %50
7,7,Elijah Mia,2023-01-12 21:41:50+00:00,https://t.co/I8RhLgOLmn Kadar konuşmadın. kot için bulunuyor. yaprak tişört Henüz mum gittiğine pantolon cesaret #kapaklı Birini iki hazır ihtiyacı değiliz! lehçesi takvimime,kadar konuşmadın kot için bulunuyor yaprak tişört henüz mum gittiğine pantolon cesaret birini iki hazır ihtiyacı değiliz lehçesi takvimime,"You haven't talked about the jeans, the leaf t -shirt is still going to the candle, the courage of one of the courage, we don't need two ready -m

In [386]:
Veri.tail(10)

,Unnamed: 0,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tweet_Toxic,Tweet_Ayrı_TR,Tür,Yoğunluk
10,10,Gençliğimiz Var,2023-01-12 21:10:55+00:00,"Bir ilin en büyük mülki idare amiri, devletin temsilcisi ""Valilik"" makamını kot pantolonla ziyaret eden bu saygısız kimdir?\n\nSüleyman Soylu'nun danışmanı ve troll koordinatörü Emin Şen!\n\nEvet devlete ve idareye saygısı olmayan kişi bakan danışmanı!\n@sgdd_asam",bir ilin en büyük mülki idare amiri devletin temsilcisi valilik makamını kot pantolonla ziyaret eden bu saygısız kimdir süleyman soylu nun danışmanı ve troll koordinatörü emin şen evet devlete ve idareye saygısı olmayan kişi bakan danışmanı,The biggest local administrative supervisor of a province is the representative of the governor's office in jeans with this disrespectful who is this disrespectful who is the advisor of Süleyman Soylu and troll coordinator Emin Şen yes,The biggest local administrative supervisor province representative governor's office jeans disrespectful disrespectful advisor Süleyman Soylu troll coordinator Emin Şen yes,Toxic değil,"['ilin', 'büyük', 'mülki', 'idare', 'amiri', 'devletin', 'temsilcisi', 'valilik', 'makamını', 'kot', 'pantolonla', 'ziyaret', 'eden', 'saygısız', 'kimdir', 'süleyman', 'soylu', 'nun', 'danışmanı', 'troll', 'koordinatörü', 'emin', 'şen', 'evet', 'devlete', 'idareye', 'saygısı', 'olmayan', 'kişi', 'bakan', 'danışmanı']",Nötr,0 - 25%
11,11,Selim,2023-01-12 20:47:40+00:00,@crnst Keramet kot da değil,keramet kot da değil,miracle is not jeans either,miracle jeans either,Toxic değil,"['keramet', 'kot']",Nötr,0 - 25%
12,12,aysima,2023-01-12 20:43:34+00:00,Taş kafamı bi aşabilip kotlardaki eskitmelere üstüne sıçılmış gözüyle bakmayı bırakınca kendime güzel bir kot almayı isterim,taş kafamı bi aşabilip kotlardaki eskitmelere üstüne sıçılmış gözüyle bakmayı bırakınca kendime güzel bir kot almayı isterim,I would like to take a nice jeans for myself when the stone can overcome my head and stop looking at the tumbled on the jeans with a shit.,I would like take nice jeans stone overcome head stop looking tumbled jeans shit.,Toxic,"['taş', 'kafamı', 'bi', 'aşabilip', 'kotlardaki', 'eskitmelere', 'üstüne', 'sıçılmış', 'gözüyle', 'bakmayı', 'bırakınca', 'kendime', 'güzel', 'kot', 'almayı', 'isterim']",Pozitif,%75 - %100
13,13,Emrah A,2023-01-12 20:34:17+00:00,@Darkwebhaber Umut veriyor kot laleleri inanmayın..,umut veriyor kot laleleri inanmayın,"Gives hope, don't believe jeans tulips","Gives hope, believe jeans tulips",Toxic değil,"['umut', 'veriyor', 'kot', 'laleleri', 'inanmayın']",Nötr,0 - 25%
14,14,Güray Aslantürk 🚚,2023-01-12 20:33:52+00:00,7 CIZIRTILI [{izmir-escort}] ILINEK KARIKATURISTLIK KAYIKCI ARDISIK TASLANMIS KOT 🍯 https://t.co/7En2hvmrk3,cizirtili izmir escort ilinek karikaturistlik kayikci ardisik taslanmis kot,cizirtili izmir escort ilinek karikaturistlik kayikci consecutively carried elevation,cizirtili izmir escort ilinek karikaturistlik kayikci consecutively carried elevation,Toxic değil,"['cizirtili', 'izmir', 'escort', 'ilinek', 'karikaturistlik', 'kayikci', 'ardisik', 'taslanmis', 'kot']",Nötr,0 - 25%
15,15,Ayşen A.,2023-01-12 20:33:46+00:00,"Bi' de oversize. Rapçi imajına bürünmeye oversize, düşük bel kot kadar uzağım. https://t.co/DJIirNZURn",bi de oversize rapçi imajına bürünmeye oversize düşük bel kot kadar uzağım,I am as far away as a low waist jeans,I far away low waist jeans,Toxic değil,"['bi', 'oversize', 'rapçi', 'imajına', 'bürünmeye', 'oversize', 'düşük', 'bel', 'kot', 'uzağım']",Nötr,%50 - %75
16,16,Hasan Girgic,2023-01-12 20:32:35+00:00,@Sevkocan2 Kot pantolon giymez çocuklarına giydirmez cemaattense gayet normal.erdoğan düşmanı olup faize mubah diyorlar.,kot pantolon giymez çocuklarına giydirmez cemaattense gayet normal erdoğan düşmanı olup faize mubah diyorlar,"They don't wear jeans, they don't dress their children, they are quite normal Erdoğan and they call it interest.","They wear jeans, dress children, quite normal E

In [387]:
#Eksik Cümle
unmasker=pipeline('fill-mask', model='xlm-roberta-base')
